# Imports

In [1]:
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from functools import partial
from rpy2.ipython import html

# Notebook settings

In [2]:
%load_ext rpy2.ipython
html.html_rdataframe = partial(html.html_rdataframe, table_class = "docutils")
rpy2.ipython.html.init_printing()
pandas2ri.activate()

# R Packages

In [3]:
importr("easyPubMed")
importr("tidyverse")

rpy2.robjects.packages.Package as a <module 'tidyverse'>

# Mine Data from PubMed

In [4]:
%%R
# Define a function that uses a query to mine PubMed publications
getAbstracts = function(query){
  # Get entrez_ids
  get_entrez_ids = get_pubmed_ids(query)
  # Get abstracts XML format
  abstractsXML = fetch_pubmed_data(pubmed_id_list = get_entrez_ids, retstart = 0, retmax = 5000, format = "xml", encoding = "UTF-8")
  # Get abstracts reformatted into a dataframe
  PubMed_df = table_articles_byAuth(pubmed_data = abstractsXML,
                                max_chars = 1000000,
                                included_authors = 'last',
                                encoding = "UTF-8"
                                )
  # Merge tite with abstract
  PubMed_df$title_abstract = paste(PubMed_df$title, PubMed_df$abstract, sep = " ")
  # Only keep inportant columns
  data_out = PubMed_df %>% 
    select(contains(c("pmid", "title_abstract")))
}

In [8]:
%%R
# Define the query
query = '("software"[MeSH Terms] OR "programming language"[All Fields]) AND ("metabolomics"[MeSH Terms] OR "metabolomics"[All Fields] OR "metabolonics"[All Fields] OR "metabolomic"[All Fields])'
query1 = '((software[MeSH Terms] OR "programming language"[All Fields]) AND (metabolomics[MeSH Terms] OR "metabolomics"[All Fields] OR "metabolonics"[All Fields] OR "metabolomic"[All Fields] OR "metabonomic"[All Fields] OR "metabonomics"[All Fields]))'
query2 = '((Metabolomic* OR metabolomic* OR metabolite* OR multiomic* OR mixomic* OR Metabolome* OR Metabolomics[Mesh] OR metabolome[Mesh])) AND (((algorithm OR toolkit) AND (code OR software)) OR “open source” OR “source code” OR “web app” OR “web application” OR “command line” OR “programming language” OR (software AND (framework OR pipeline OR tool OR package OR suite OR workflow)) OR github OR gitlab OR sourceforge OR Bioconductor OR biopython OR biojava OR bioruby OR “Computing Methodologies”[Mesh])'
# Call the function
minedText = getAbstracts(query=query1)

R[write to console]: Processing PubMed data 
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to console]: .
R[write to c

In [9]:
# Check that we have the title+abstracts as a column and the pmid as a column
%R head(minedText)

,pmid,title_abstract
1,34024419,Novel acquisition strategies for metabolomics ...
2,33777285,A framework for stability-based module detecti...
3,33611117,Identification of the metabolites regulated in...
4,33607938,CANTARE: finding and visualizing network-based...
5,33483720,Cognitive analysis of metabolomics data for sy...
6,33467846,A Practical Guide to Metabolomics Software Dev...


In [10]:
%R nrow(minedText)

array([985], dtype=int32)

In [11]:
# Save the data
%R write.csv(minedText, file="Data/minedText_query1.2.csv", row.names = F)